In [0]:
from pandas_datareader import data as dr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

DATA

In [0]:
start_date = '2000-01-01'
end_date = '2019-01-05'

In [0]:
def getTicks(items):
  symbols = []
  for i in items:
    try:
      data = dr.get_data_yahoo(i, start_date, end_date)
    except Exception as e:
      print("there was a problem with {}".format(i))
    else:
      symbols.append(i)
  return symbols

In [0]:
sample_ticks = [
            'NVS'                
]

adj = getTicks(sample_ticks)

In [0]:
df = dr.get_data_yahoo(adj, start_date, end_date)

In [6]:
df.head()

Attributes,High,Low,Open,Close,Volume,Adj Close
Symbols,NVS,NVS,NVS,NVS,NVS,NVS
Date,,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,133200.0,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,243200.0,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,144600.0,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,579200.0,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,89900.0,18.152739


In [0]:
prices = df.loc[:,('Adj Close',slice(None))]


In [0]:
prices.columns = prices.columns.droplevel(0)

In [9]:
prices.head()

Symbols,NVS
Date,
2000-01-03,17.756123
2000-01-04,17.420527
2000-01-05,17.496799
2000-01-06,17.878160
2000-01-07,18.152739


In [10]:
p = prices.loc['2000-01-01':'2000-04-01']
p.tail()

Symbols,NVS
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [11]:
pip install tbats

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 1.1MB 11.9MB/s 


In [12]:
pip install pyramid.arima

     |████████████████████████████████| 604kB 4.9MB/s 


In [13]:
pip install prophet

  Created wheel for prophet: filename=prophet-0.1.1-cp36-none-any.whl size=12168 sha256=07ce9c25087d76e7a6b7def9e58233749050277473fbb90a605c2c0dd7707f02
  Stored in directory: /root/.cache/pip/wheels/77/3e/f3/1c536bf1f871f818686e7fbf31cab18d52787a72dea8640756
Successfully built prophet


In [0]:
from tbats import TBATS
from pmdarima import auto_arima
from fbprophet import Prophet

MODELS:
Forecasting 30-90 Days

Simple Average DONE

In [0]:
## Error metric - MAPE

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [0]:
def sam(ts):
    avg = ts.mean()
    return(avg)

In [0]:
def rolling_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        prediction = sam(local_train_timeSeries[local_ticker])#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)


In [35]:
rolling_cv(p)

,ticker,MAPE
0,NVS,6.481251


AutoArima DONE

In [0]:
def autoarima(ts,idx):
  
  model = auto_arima(ts, start_p=1, start_q=1, d=0,max_p=5,max_d= 2, max_q=5, error_action='ignore',suppress_warnings=True)
  #trace=True,
  model.fit(ts)

  forecast = model.predict(n_periods=1)# n_periods
  fc = pd.DataFrame(forecast,index = idx,columns=['Prediction'])# valid.index => idx, index must be called with a collection of some kind
  return (fc)

In [0]:
def aa_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = autoarima(local_train_timeSeries[local_ticker],df.loc[df.index == crossDate].index)#****
        #print(prediction.values)
        df.loc[df.index == crossDate, 'prediction'] = prediction.values
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]   

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [61]:
p.tail()

Symbols,NVS
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [26]:
aa_cv(p)

,ticker,MAPE
0,NVS,1.499655


Fourier Transform for AutoArima DONE

In [54]:
test = df.loc[:,(['Open', 'Close','High','Adj Close','Low'],slice(None))]
test.head()

Attributes,High,Low,Open,Close,Adj Close
Symbols,NVS,NVS,NVS,NVS,NVS
Date,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,18.152739


In [0]:
test.columns = test.columns.droplevel(1)

In [0]:
def ftransform(data):
    data = data.reset_index()
    data_FT = data[['Date', 'Adj Close']]
    close_fft = np.fft.fft(np.asarray(data_FT['Adj Close'].tolist()))
    close_fft = np.fft.ifft(close_fft)
    #close_fft
    fft_df = pd.DataFrame({'fft':close_fft})
    #print(fft_df.tail())
    fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
    fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
    fft_list = np.asarray(fft_df['fft'].tolist())
    fft_list_m10= np.copy(fft_list) 
    fft_list_m10[100:-100]=0
    # 100 components
    data['Fourier'] = pd.DataFrame(fft_list_m10).apply(lambda x: np.abs(x))
    data = data.set_index('Date')
    data = data[['Fourier']]
    print(data[-10:-1])
    #dataset['absolute'] = dataset['Fourier'].apply(lambda x: np.abs(x))
    return data

In [70]:
dta = ftransform(test)

Attributes    Fourier
Date                 
2018-12-20  73.109093
2018-12-21  72.345284
2018-12-24  71.598831
2018-12-26  73.577797
2018-12-27  72.970222
2018-12-28  74.003105
2018-12-31  74.480492
2019-01-02  72.961540
2019-01-03  73.551765


In [71]:
dta  = dta.loc['2000-01-01':'2000-04-01']
dta.tail()

Attributes,Fourier
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [72]:
aa_cv(dta)

,ticker,MAPE
0,Fourier,1.499665


TBATS DONE

In [0]:
# Fit the model
def tbats(ts):
  estimator = TBATS(seasonal_periods=(7, 30),use_box_cox=False)
  model = estimator.fit(ts)

  model_forecast = model.forecast(steps=1)
  return model_forecast

In [0]:
def tbats_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = tbats(local_train_timeSeries[local_ticker])#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [31]:
tbats_cv(p)
# about 10 minutes to complete

,ticker,MAPE
0,NVS,3.126702


FB Prophet DONE

In [0]:
def prophet(df,local_ticker):
  fb = df.reset_index()
  fb = fb.rename(columns = {'Date':'ds', local_ticker:'y'})
  m = Prophet()
  m.fit(fb)
  future = m.make_future_dataframe(periods=1)
  forecast = m.predict(future)
  return forecast.iloc[-1][-1]

In [0]:
def fb_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = prophet(local_train_timeSeries[local_ticker],local_ticker)#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [38]:
fb_cv(p)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 24.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this

,ticker,MAPE
0,NVS,36.877404


LSTM DONE

In [75]:
pip install tensorflow

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [82]:
dataset = test.copy()
dataset.head()

Attributes,High,Low,Open,Close,Adj Close
Date,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,18.152739


In [0]:
# IMPORTING DATASET 
dataset = dataset.reindex(index = dataset.index[::-1])

In [0]:
# CREATING OWN INDEX FOR FLEXIBILITY
obs = np.arange(1, len(dataset) + 1, 1)

In [0]:
# TAKING DIFFERENT INDICATORS FOR PREDICTION
OHLC_avg = p.mean(axis = 1)

In [0]:
# PREPARATION OF TIME SERIES DATASE
OHLC_avg = np.reshape(OHLC_avg.values, (len(OHLC_avg),1)) 
scaler = MinMaxScaler(feature_range=(0, 1))
OHLC_avg = scaler.fit_transform(OHLC_avg)

In [0]:
# TRAIN-TEST SPLIT
train_OHLC = int(len(OHLC_avg) * 0.75)
test_OHLC = len(OHLC_avg) - train_OHLC
train_OHLC, test_OHLC = OHLC_avg[0:train_OHLC,:], OHLC_avg[train_OHLC:len(OHLC_avg),:]

In [0]:
# FUNCTION TO CREATE 1D DATA INTO TIME SERIES DATASET
def new_dataset(dataset, step_size):
  data_X, data_Y = [], []
  limit = len(dataset)-step_size-1

  for i in range(limit):
    j = i + step_size
    #a = dataset.iloc[i:j, 0]
    a = dataset[i:j, 0]
    #print(a)
    data_X.append(a)
    #b = dataset.iloc[j, 0]
    b = dataset[j, 0]
    #print(b)
    data_Y.append(b)
  return np.asarray(data_X), np.asarray(data_Y)
  #return np.asarray(data_X), np.asarray(data_Y)

In [0]:
# TIME-SERIES DATASET (FOR TIME T, VALUES FOR TIME T+1)
trainX, trainY = new_dataset(train_OHLC, 1)
testX, testY = new_dataset(test_OHLC, 1)

In [0]:
# RESHAPING TRAIN AND TEST DATA
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
step_size = 1

In [0]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(32, input_shape=(1, step_size), return_sequences = True))
model.add(LSTM(16))
model.add(Dense(1))
model.add(Activation('linear'))

In [96]:
# MODEL COMPILING AND TRAINING
model.compile(loss='mean_squared_error', optimizer='adagrad',metrics = ['mae']) 
# compare mae, adam, adagrad
model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=2)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where









Epoch 1/50


 - 2s - loss: 0.1101 - mean_absolute_error: 0.2561
Epoch 2/50
 - 0s - loss: 0.0428 - mean_absolute_error: 0.1705
Epoch 3/50
 - 0s - loss: 0.0351 - mean_absolute_error: 0.1647
Epoch 4/50
 - 0s - loss: 0.0302 - mean_absolute_error: 0.1523
Epoch 5/50
 - 0s - loss: 0.0262 - mean_absolute_error: 0.1407
Epoch 6/50
 - 0s - loss: 0.0224 - mean_absolute_error: 0.1286
Epoch 7/50
 - 0s - loss: 0.0192 - mean_absolute_error: 0.1176
Epoch 8/50
 - 0s - loss: 0.0167 - mean_absolute_error: 0.1077
Epoch 9/50
 - 0s - loss: 0.0146 - mean_absolute_error: 0.0991
Epoch 10/50
 - 0s - loss: 0.0127 - mean_absolute_error: 0.0908
Epoch 11/50
 - 0s - loss: 0.0113 - mean_absolute_error: 0.0854
Epoch 12/50
 - 0s - loss: 0.0102 - mean_absolute_error: 0.0791
Epoch 13/50
 - 0s - loss: 0.0094 - mean_absolute_error: 0.0771
Epoch 14/50
 - 0s - loss: 0.0088 - mean_absolute_error: 0.0737
Epoch 15/50
 - 0s - loss: 0.0082 - mean_absolute_error: 0.0700
Epoch 16/50
 - 0s - loss: 0.0078 - mean_absolute_error: 0.0687
Epoch 17/50


In [99]:
mae = model.evaluate(testX, testY, batch_size=16)
print('MAE:', mae)

14/14 [==============================] - 0s 282us/step
MAE: [0.004900902509689331, 0.05640620365738869]
